In [66]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Import API key
from api_keys import g_key

In [67]:
#Store Part I results into DataFrame
#Load the csv exported in Part I to a DataFrame
csv = "output_data/cities.csv"
cities_df = pd.read_csv(csv,encoding="UTF-8")
cities_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed
0,0,Hilo,90,US,1579316804,73,19.73,-155.09,75.20,16.11
1,1,Pacific Grove,1,US,1579316589,81,36.62,-121.92,50.00,6.93
2,2,Puerto Ayora,27,EC,1579316807,66,-0.74,-90.35,77.00,1.01
3,3,Arraial do Cabo,100,BR,1579316808,78,-22.97,-42.03,75.56,18.59
4,4,Klaksvik,100,FO,1579316762,80,62.23,-6.59,39.20,23.04


In [68]:
gmaps.configure(api_key=g_key)

In [69]:
coordinates = cities_df[["Latitude", "Longitude"]]
humidity = cities_df["Humidity"].astype(float)
figure_layout = {
     'width' : '1000px',
     'height':'400px',
     'border':'1px solid black',
     'padding':'1px',
     'margin':'0 auto 0 auto'
}

fig = gmaps.figure()
heatmaps = gmaps.heatmap_layer(coordinates, weights=humidity,
                                   dissipating=False, max_intensity=10,
                                   point_radius=2)
fig.add_layer(heatmaps)
fig

Figure(layout=FigureLayout(height='420px'))

In [88]:
# Narrow down the DataFrame to find your ideal weather condition. For example:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. 
# You want to be sure the weather is ideal
temp = (cities_df["Max Temperature"] > 70) & (cities_df["Max Temperature"] <+ 80)
windy = (cities_df["Wind Speed"] < 10)
cloudless = cities_df["Cloudiness"] == 0

hotel_df = cities_df[temp & windy & cloudless]

In [71]:
hotel_df["Hotel Name"] = ""
hotel_df

/Users/samstruthers/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temperature,Wind Speed,Hotel Name
59,59,Esim,0,GH,1579316876,92,4.87,-2.24,77.13,2.71,
137,137,Port Blair,0,IN,1579316974,88,11.67,92.75,78.80,2.24,
149,149,Champasak,0,LA,1579316988,73,14.75,106.00,71.60,2.24,
169,169,Prachuap Khiri Khan,0,TH,1579316624,75,11.82,99.78,79.86,0.60,
205,205,Mitu,0,CO,1579316631,94,1.20,-70.17,72.05,0.65,
206,206,El Bagre,0,CO,1579316632,96,7.59,-74.81,75.00,1.97,
224,224,Collie,0,AU,1579316635,22,-33.37,116.15,79.00,7.43,
282,282,Meiktila,0,MM,1579316648,42,20.87,95.87,73.26,1.10,
294,294,Tura,0,IN,1579317165,55,25.52,90.22,72.32,1.79,
321,321,Syriam,0,MM,1579316657,60,16.77,96.24,75.20,4.70,


In [86]:
#Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.
hotel_names = []
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}
for index, row in hotel_df.iterrows():
    latitude = row["Latitude"]
    longitude = row["Longitude"]
    params["location"] = f"{latitude},{longitude}"
    query_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(query_url, params=params).json()
   
    try:
        hotel_names.append(response['results'][0]['name'])
        print(response['results'][0]['name'])
    except IndexError:
        hotel_names.append(np.nan)
        print("Could Not Find")
    continue    

Axim Beach Hotel
Sinclairs Bayview Port Blair
Kingfisher Ecolodge
Prachuap Grand Hotel
Could Not Find
Hotel Laureles
Premier Hotel
A1 Motel
Hotel Polo Orchid
WH Hotel
Could Not Find
Pousada Rio
Noktel Resort Hotel
Villavela Hotel Boutique
Dumont Palace Hotel
Hôtel La residence
Could Not Find


In [98]:
hotel_df["Hotel Name"] = hotel_names
hotel_df["Hotel Name"].dropna()
hotel_df.to_csv('output_data/hotel_list.csv')

/Users/samstruthers/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [99]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [100]:
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))